In [ ]:
!pip install git+https://github.com/ybaktir/networkx-neo4j

In [ ]:
!pip install neo4j

In [ ]:
# !pip freeze

In [12]:
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', 100) # default 50, the maximum width in characters of a column
pd.set_option('display.max_columns', 40)   # default 20, the maximum amount of columns in view 
pd.set_option('display.max_rows', 60)      # default 60, the maximum amount of rows in view

# graph neo4j packages
from py2neo import Graph, Node, Relationship
import nxneo4j as nx
from neo4j import GraphDatabase

# operating system packages
import platform
import psutil
import os

In [13]:
# load graph connection instance (https://sandbox.neo4j.com/)
port      = "7687"
user      = "neo4j"
passw     = "07xLgl5sy7uUazqtWh1QvuRHSTIkkUZ_kMz5Hqg8Twc"
db_name   = "neo4j"
uri       = "neo4j+s://2abbd1fc.databases.neo4j.io"

# py2neo instance
graph   = Graph(uri, auth=(user, passw))

# nxneo4j instance
driver  = GraphDatabase.driver(uri, auth=(user, passw))
G       = nx.DiGraph(driver) # directed graph

# neo4j database version
graph.call.dbms.components()

name,versions,edition
Neo4j Kernel,['4.4-aura'],enterprise


In [24]:
print(graph.__dict__)

{'service': <GraphService uri='neo4j+s://2abbd1fc.databases.neo4j.io:7687' secure=True user_agent=None>, '__name__': None, 'schema': <py2neo.database.Schema object at 0x7fb5a9acd550>, '_procedures': <py2neo.cypher.proc.ProcedureLibrary object at 0x7fb5a9acd310>}


In [14]:
# list of files
SAS_url   = "https://datalake28042022.blob.core.windows.net/"
signkey   = "?sv=2020-02-10&st=2022-04-30T13%3A49%3A23Z&se=2022-05-29T13%3A49%3A00Z&sr=d&sp=racwdl&sig=JD%2BAFdAqKPT%2FZaUJad9dA%2FkhbQhzDjhIGEeJwGxPbdk%3D&sdd=1"

come_from_url = SAS_url + "datalake/silver/come_from.csv" + signkey
complete_cleaned_url = SAS_url + "datalake/silver/complete_cleaned.csv" + signkey
date_url = SAS_url + "datalake/silver/date.csv" + signkey
has_product_url = SAS_url + "datalake/silver/has_product.csv" + signkey
located_in_url = SAS_url + "datalake/silver/located_in.csv" + signkey

In [ ]:
pd.read_csv(date_url)

In [ ]:
pd.read_csv(date_url).drop_duplicates()

## create nodes

In [15]:
G.delete_all()

In [ ]:
graph.run("CREATE CONSTRAINT ON (co:Company) ASSERT co.CompanyName IS UNIQUE")

In [16]:
%%time
query = """
USING PERIODIC COMMIT 10000
LOAD CSV WITH HEADERS FROM '"""+complete_cleaned_url+"""' AS line FIELDTERMINATOR ','
MERGE (co:Company {CompanyName:(line.CompanyName)})
"""
display(graph.run(query).stats())

{'labels_added': 880, 'nodes_created': 880, 'properties_set': 880}

CPU times: user 1.9 ms, sys: 4.06 ms, total: 5.95 ms
Wall time: 562 ms


In [ ]:
graph.run("CREATE CONSTRAINT ON (pr:Product) ASSERT pr.ProductName IS UNIQUE")

In [17]:
%%time
query = """
USING PERIODIC COMMIT 10000
LOAD CSV WITH HEADERS FROM '"""+date_url+"""' AS line FIELDTERMINATOR ','
MERGE (pr:Product {ProductName:(line.ProductName)})
"""
display(graph.run(query).stats())

{'labels_added': 30721, 'nodes_created': 30721, 'properties_set': 30721}

CPU times: user 11.9 ms, sys: 259 µs, total: 12.2 ms
Wall time: 1.29 s


In [7]:
graph.run("CREATE CONSTRAINT ON (cy:Country) ASSERT cy.CountryName IS UNIQUE")

(No data)

In [18]:
%%time
query = """
USING PERIODIC COMMIT 10000
LOAD CSV WITH HEADERS FROM '"""+complete_cleaned_url+"""' AS line FIELDTERMINATOR ','
MERGE (cy:Country {CountryName:(line.CountryName)})
"""
display(graph.run(query).stats())

{'labels_added': 26, 'nodes_created': 26, 'properties_set': 26}

CPU times: user 5.14 ms, sys: 0 ns, total: 5.14 ms
Wall time: 338 ms


## relationship

In [19]:
%%time
query = """
USING PERIODIC COMMIT 10000
LOAD CSV WITH HEADERS FROM '"""+located_in_url+"""' AS line FIELDTERMINATOR ','
MATCH (co:Company {CompanyName:(line.CompanyName)})
MATCH (cy:Country {CountryName:(line.CountryName)})
MERGE (co)-[:LOCATED_IN]->(cy)
"""
display(graph.run(query).stats())

{'relationships_created': 888}

CPU times: user 21.4 ms, sys: 7.62 ms, total: 29 ms
Wall time: 266 ms


In [20]:
%%time
query = """
USING PERIODIC COMMIT 10000
LOAD CSV WITH HEADERS FROM '"""+has_product_url+"""' AS line FIELDTERMINATOR ','
MATCH (co:Company {CompanyName:(line.CompanyName)})
MATCH (pr:Product {ProductName:(line.ProductName)})
MERGE (co)-[:HAS_PRODUCT]->(pr)
"""
display(graph.run(query).stats())

{'relationships_created': 31272}

CPU times: user 5.71 ms, sys: 317 µs, total: 6.03 ms
Wall time: 2.09 s


In [21]:
%%time
query = """
USING PERIODIC COMMIT 10000
LOAD CSV WITH HEADERS FROM '"""+come_from_url+"""' AS line FIELDTERMINATOR ','
MATCH (pr:Product {ProductName:(line.ProductName)})
MATCH (cy:Country {CountryName:(line.CountryName)})
MERGE (pr)-[:COME_FROM]->(cy)
"""
display(graph.run(query).stats())

{'relationships_created': 30731}

CPU times: user 317 µs, sys: 4.91 ms, total: 5.23 ms
Wall time: 1.45 s


## queries

In [ ]:
query = """
MATCH (w:Winery)-[:FROM_PROVENCE]->(p:Province)-[:PROVINCE_COUNTRY]->(c:Country)
RETURN c.name AS Country, count(DISTINCT w) AS totalNrWineries
ORDER BY totalNrWineries DESC LIMIT 10
"""
graph.run(query).to_data_frame()

In [2]:
import datetime
import pandas as pd
from py2neo import Graph

def get_all_products():
    """
    Returns all products from the database
    """
    user      = "neo4j"
    passw     = "07xLgl5sy7uUazqtWh1QvuRHSTIkkUZ_kMz5Hqg8Twc"
    uri       = "neo4j+s://2abbd1fc.databases.neo4j.io"

    graph = Graph(uri, auth=(user, passw))
    query = """
    MATCH (n:Country)
    RETURN n LIMIT 25
    """
    return graph.run(query).to_data_frame().to_string()

                                    n
0          {'CountryName': 'Austria'}
1          {'CountryName': 'Belgium'}
2         {'CountryName': 'Bulgaria'}
3           {'CountryName': 'Cyprus'}
4   {'CountryName': 'Czech Republic'}
5          {'CountryName': 'Germany'}
6          {'CountryName': 'Denmark'}
7          {'CountryName': 'Estonia'}
8            {'CountryName': 'Spain'}
9          {'CountryName': 'Finland'}
10          {'CountryName': 'France'}
11          {'CountryName': 'Greece'}
12         {'CountryName': 'Croatia'}
13         {'CountryName': 'Hungary'}
14         {'CountryName': 'Ireland'}
15           {'CountryName': 'Italy'}
16       {'CountryName': 'Lithuania'}
17          {'CountryName': 'Latvia'}
18     {'CountryName': 'Netherlands'}
19          {'CountryName': 'Norway'}
20          {'CountryName': 'Poland'}
21        {'CountryName': 'Portugal'}
22         {'CountryName': 'Romania'}
23          {'CountryName': 'Sweden'}
24        {'CountryName': 'Slovenia'}


In [5]:
!pip freeze

asttokens==2.0.5
azure-core==1.23.1
azure-storage-blob==12.11.0
backcall==0.2.0
certifi==2021.10.8
cffi==1.15.0
charset-normalizer==2.0.12
cramjam==2.5.0
cryptography==37.0.1
debugpy==1.6.0
decorator==5.1.1
entrypoints==0.4
executing==0.8.3
fastparquet==0.8.1
fsspec==2022.3.0
fuzzywuzzy==0.18.0
idna==3.3
interchange==2021.0.4
ipykernel==6.13.0
ipython==8.2.0
ipython-cypher==0.2.6
isodate==0.6.1
jedi==0.18.1
jupyter-client==7.3.0
jupyter-core==4.10.0
matplotlib-inline==0.1.3
monotonic==1.6
msrest==0.6.21
neo4j==4.4.3
neo4j-driver==4.4.3
neo4jrestclient==2.1.1
nest-asyncio==1.5.5
networkx-neo4j @ git+https://github.com/ybaktir/networkx-neo4j@97dc9563bf992ea9714cbdb99cb9e6a41c7cce65
numpy==1.22.3
oauthlib==3.2.0
packaging==21.3
pandas==1.4.2
pansi==2020.7.3
parso==0.8.3
pexpect==4.8.0
pickleshare==0.7.5
prettytable==3.2.0
prompt-toolkit==3.0.29
psutil==5.9.0
ptyprocess==0.7.0
pure-eval==0.2.2
py2neo==2021.2.3
pyarrow==7.0.0
pycparser==2.21
Pygments==2.12.0
pyparsing==3.0.8
python-dateutil